In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# 进行模式输出插值之后的mask操作，到CN05.1的中国区域
2021.12.20

降水和气温放在同一个文件中，只是变量名的不同

插值后 lat lon 的格点数目全部一致。Time因为原始数据问题有一些不同

MASK数组取一个2D的 lat x lon数组，np.where自动进行时间维度上的广播

# 降水处理 数据读入

目标处理：1991-2015年数据

In [2]:
path_in = "/raid61/LS4P-TPEMIP/TEMP_DATA/precip_regrid/"
ds_in = {}
dir_in ="/raid61/LS4P-TPEMIP/REFERENCE/CN05.1/"
ds_in['obs'] = xr.open_dataset(dir_in + "sel_CN05.1_Pre_1961_2018_daily_025x025.nc").rename({"pre":"pr"})
# ds_in[''] = xr.open_dataset(path_in + "")
ds_in['CWRF'] = xr.open_dataset(path_in + "CWRF_pre.nc") # 缺少逐日数据，只有多种尺度的平均态数据
ds_in['JAMSTECWRF'] = xr.open_dataset(path_in + "JAMSTECWRF_pre.nc") # 缺少4月
ds_in['NJU_LS4P_B'] = xr.open_dataset(path_in + "NJU_LS4P_B_pre.nc")
ds_in['NJU_LS4P_C'] = xr.open_dataset(path_in + "NJU_LS4P_C_pre.nc")
ds_in['NJU_Output-E'] = xr.open_dataset(path_in + "NJU_Output-E_pre.nc") # 缺少2015年
ds_in['SYS'] = xr.open_dataset(path_in + "SYS_pre.nc")
ds_in['Yang-ZhouX'] = xr.open_dataset(path_in + "Yang-ZhouX_pre.nc")
ds_in['YSU'] = xr.open_dataset(path_in + "YSU_pre.nc")

/home/yycheng/miniconda3/envs/proplot1006/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [6]:
# 稍微处理一下变量名，用于后续的处理
for single_mod in ds_in:
    print(single_mod)
    list_vars = list(ds_in[single_mod].variables)
    print(list_vars)
#     print(ds_in[single_mod].coords)
    if ('Time' in list_vars):
        print('----- rename Time>>>time')
        ds_in[single_mod] = ds_in[single_mod].rename({"Time":"time"})
    if ('pr' not in list_vars):
        print('------ rename ???pre???>>>pr')
        ds_in[single_mod] = ds_in[single_mod].rename({"dailypre":"pr"})
    if (ds_in[single_mod].time.shape[0] !=3825):
        print("**********time error in " + single_mod + " !" + " time length:" + str(ds_in[single_mod].time.shape[0]) )

obs
['time', 'lon', 'lat', 'pr']
CWRF
['lon', 'lat', 'time', 'pr']
**********time error in CWRF ! time length:12
JAMSTECWRF
['lon', 'lat', 'time', 'pr']
**********time error in JAMSTECWRF ! time length:3075
NJU_LS4P_B
['lon', 'lat', 'time', 'pr']
NJU_LS4P_C
['lon', 'lat', 'time', 'pr']
NJU_Output-E
['lon', 'lat', 'time', 'pr']
SYS
['lon', 'lat', 'Time', 'dailypre']
----- rename Time>>>time
------ rename ???pre???>>>pr
Yang-ZhouX
['lon', 'lat', 'Time', 'dailypre']
----- rename Time>>>time
------ rename ???pre???>>>pr
YSU
['lon', 'lat', 'time', 'pr']


## MASK操作，以及数据写出

In [71]:
mask_temp2d = ~np.isnan( ds_in['obs']['pr'][0,:,:].values.copy() )
for single_mod in ds_in:
    if (single_mod == 'obs'):
        continue
    print("-----dealing mask on : " + single_mod)
    path_out = "/raid61/LS4P-TPEMIP/TEMP_DATA/precip_mask/"
    mask_target = ds_in[single_mod].copy()
    mask_result_temp = xr.where(mask_temp2d, mask_target, np.nan)
    mask_result_temp.to_netcdf(path_out + single_mod + "_pre.nc")
    mask_result_temp.close()
    del(mask_result_temp)

-----dealing mask on : CWRF
-----dealing mask on : JAMSTECWRF
-----dealing mask on : NJU_LS4P_B
-----dealing mask on : NJU_LS4P_C
-----dealing mask on : NJU_Output-E
-----dealing mask on : SYS
-----dealing mask on : Yang-ZhouX
-----dealing mask on : YSU


# 气温处理 数据读入

目标处理：1991-2015年数据

为了改变量名方便处理为了DataArray

In [6]:
path_in = "/raid61/LS4P-TPEMIP/TEMP_DATA/t2m_regrid/"
ds_in = {}
dir_in ="/raid61/LS4P-TPEMIP/REFERENCE/CN05.1/"
ds_in['obs'] = xr.open_dataset(dir_in + "sel_CN05.1_Tm_1961_2018_daily_025x025.nc")['tm'].rename("t2m")
# ds_in[''] = xr.open_dataset(path_in + "")
ds_in['CWRF'] = xr.open_dataset(path_in + "CWRF_t2m.nc")['t2m'] # 缺少逐日数据，只有多种尺度的平均态数据
ds_in['JAMSTECWRF'] = xr.open_dataset(path_in + "JAMSTECWRF_t2m.nc")['t2m'] # 缺少4月
ds_in['NJU_LS4P_B'] = xr.open_dataset(path_in + "NJU_LS4P_B_t2m.nc")['tas'][:,0,:,:].rename("t2m")
ds_in['NJU_LS4P_C'] = xr.open_dataset(path_in + "NJU_LS4P_C_t2m.nc")['tas'][:,0,:,:].rename("t2m")
ds_in['NJU_Output-E'] = xr.open_dataset(path_in + "NJU_Output-E_t2m.nc")['tas'][:,0,:,:].rename("t2m") # 缺少2015年
ds_in['SYS'] = xr.open_dataset(path_in + "SYS_t2m.nc")['T2'].rename("t2m")
ds_in['Yang-ZhouX'] = xr.open_dataset(path_in + "Yang-ZhouX_t2m.nc")['T2'].rename("t2m")
ds_in['YSU'] = xr.open_dataset(path_in + "YSU_t2m.nc")['t2m']

/home/yycheng/miniconda3/envs/proplot1006/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [8]:
# 稍微处理一下变量名，用于后续的处理
for single_mod in ds_in:
    print(single_mod)
    list_vars = list(ds_in[single_mod].coords)
    print(list_vars)
#     print(ds_in[single_mod].variables)
#     print(ds_in[single_mod].coords)
    if ('Time' in list_vars):
        print('----- rename Time>>>time')
        ds_in[single_mod] = ds_in[single_mod].rename({"Time":"time"})
#     if (ds_in[single_mod].time.shape[0] !=3825):
#         print("**********time error in " + single_mod + " !" + " time length:" + str(ds_in[single_mod].time.shape[0]) )

obs
['time', 'lon', 'lat']
CWRF
['lon', 'lat', 'time']
JAMSTECWRF
['lon', 'lat', 'time']
NJU_LS4P_B
['lon', 'lat', 'm2', 'time']
NJU_LS4P_C
['lon', 'lat', 'm2', 'time']
NJU_Output-E
['lon', 'lat', 'm2', 'time']
SYS
['lon', 'lat', 'time']
Yang-ZhouX
['lon', 'lat', 'time']
YSU
['lon', 'lat', 'time']


## MASK操作，以及数据写出

In [14]:
mask_temp2d = ~np.isnan( ds_in['obs'][0,:,:].values.copy() )
for single_mod in ds_in:
    if (single_mod == 'obs'):
        continue
    print("-----dealing mask on : " + single_mod)
    path_out = "/raid61/LS4P-TPEMIP/TEMP_DATA/t2m_mask/"
    mask_target = ds_in[single_mod].copy()
    mask_result_temp = xr.where(mask_temp2d, mask_target, np.nan)
    mask_result_temp.to_netcdf(path_out + single_mod + "_t2m.nc")
    mask_result_temp.close()
    del(mask_result_temp)

-----dealing mask on : CWRF
-----dealing mask on : JAMSTECWRF


/home/yycheng/miniconda3/envs/proplot1006/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-01-01 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-01-01 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


-----dealing mask on : NJU_LS4P_B
-----dealing mask on : NJU_LS4P_C
-----dealing mask on : NJU_Output-E
-----dealing mask on : SYS
-----dealing mask on : Yang-ZhouX
-----dealing mask on : YSU
